In [1]:
%tensorflow_version 2.2
!pip install twilio
!pip install tqdm
!pip install jarvis-md
!pip install nibabel
import os
import glob
import numpy as np
import tensorflow as tf
from tensorflow.keras import Input, Model, models, layers
from tensorflow.keras.utils import normalize
import matplotlib.pyplot as plt
%matplotlib inline
from google.colab import files
from twilio.rest import Client
import tqdm
import time
from jarvis.train import custom
from nibabel import load
from scipy.ndimage import rotate

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.2`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:

def create_ID_dict(basepath):
  dictionary = {}
  mask_list =[]
  image_list =[]

  for file in glob.glob(basepath+'/**/**.nii.gz', recursive=True):
    if 'data.nii' in file:
      mask_list.append(file)
    elif 'label.nii' in file:
      image_list.append(file)

  
  dictionary.update({"image":image_list})
  dictionary.update({"mask":mask_list})
  return dictionary


def dataset_gen():
    image_list= [] 
    mask_list = []
    dictionary = create_ID_dict(PATH_tr)
    mask= True
    if mask == True:
        n = 0

        for data_path, label_path in zip(dictionary['image'],dictionary['mask']):
            image = load(data_path)
            image_array = image.get_fdata().astype('float32')#dtype float 64 -> float32
            image_array = rotate(image_array, 90, axes=(0,2), reshape=True)
            #normalize?

            
            mask = load(label_path) #preprocessing has happened befoer hand
            mask_array = np.array(mask.dataobj) #dtype int16
            mask_array = rotate(mask_array, 90, axes=(0,2), reshape=True)
        

            for slice_image, slice_mask in zip(image_array,mask_array):
                slice_image = np.expand_dims(slice_image, axis =2 )
                print("slice_image shape: ", slice_image.shape)

                slice_mask = np.expand_dims(slice_mask, axis =2 )
                print("slice_mask shape: ", slice_mask.shape)
                
                yield tf.convert_to_tensor(slice_image), tf.convert_to_tensor(slice_mask)

                ''' image_list.append(slice_image)
                mask_list.append(slice_mask)



            n+=1
            print("{} images have been loaded.".format(str(n)+'/'+str(len(dictionary['image']))))

        return image_list, mask_list'''
    else:
        print("option not possible yet")

In [4]:
'''Predetermined Variables'''
PATH_tr = '/content/drive/My Drive/StructSeg2019 Thoracic Data'
model_name = "***"
save_path = '***'+model_name

In [5]:
image = load('/content/drive/My Drive/StructSeg2019 Thoracic Data/1/data.nii.gz')
mask = load('/content/drive/My Drive/StructSeg2019 Thoracic Data/1/label.nii.gz')

'''Task3_Thoracic_OAR.zip: Organ-at-risk segmentation from chest CT scans. Same format as above. 
The 6 annotated OARs contain Lung_L,Lung_R,Heart,Esophagus,Trachea,Spinal_Cord, 
corresponding to the label 1 to 6 in the annotation file.'''

'Task3_Thoracic_OAR.zip: Organ-at-risk segmentation from chest CT scans. Same format as above. \nThe 6 annotated OARs contain Lung_L,Lung_R,Heart,Esophagus,Trachea,Spinal_Cord, \ncorresponding to the label 1 to 6 in the annotation file.'

In [26]:
tr_dict = create_ID_dict(PATH_tr)
#print(tr_dict)
time_start = time.time()
#image_list, mask_list = dataset_gen(tr_dict,mask=True)
dataset = tf.data.Dataset.from_generator(dataset_gen,output_types = (tf.float32, tf.int16), output_shapes=(tf.TensorShape([None,512,512,1]), tf.TensorShape([None,512,512,1])))
print("Loading into Tensors")
#image_list = tf.convert_to_tensor(image_list)
#mask_list = tf.convert_to_tensor(mask_list)

#image_list = iter(image_list)
#mask_list = iter(mask_list)

'''Dr. Chang is making a Jarvis Generator of Dataset. Will be ready on Sunday'''
time_end = time.time()
send_text("Data Generation Completed in {} min.".format((time_end-time_start)/60))

Loading into Tensors


In [22]:
for slice, mask in dataset.as_numpy_iterator():
    print(slice.shape, mask.shape)

slice_image shape:  (512, 512, 1)
slice_mask shape:  (512, 512, 1)


KeyboardInterrupt: ignored

In [29]:
def NN():
    '''Model Creation'''

    #Define model#
    # Define kwargs dictionary#
    kwargs = {
    'kernel_size': (3,3),
    'padding': 'same',
    'data_format': 'channels_last',
    'bias_initializer':'zeros'} #zeros, ones, golorit_uniform

    #Define lambda functions#
    conv = lambda x, filters, strides : layers.Conv2D(filters=filters, strides=strides, **kwargs)(x)
    norm = lambda x : layers.BatchNormalization()(x)
    relu = lambda x : layers.LeakyReLU()(x)

    #Define stride-1, stride-2 blocks#
    conv1 = lambda filters, x : relu(norm(conv(x, filters, strides=1)))
    conv2 = lambda filters, x : relu(norm(conv(x, filters, strides=2)))

    #Define single transpose#
    tran = lambda x, filters, strides : layers.Conv2DTranspose(filters=filters, strides=strides, **kwargs)(x)

    #Define transpose block#
    tran2 = lambda filters, x : relu(norm(tran(x, filters, strides=2)))
    concat = lambda a, b : layers.Concatenate(axis=3)([a, b])

    #Define Dense Block#
    def dense_block(filters,input):
        l1 = conv1(filters, input)
        c1 = concat(input,l1)
        l2 = conv1(filters,c1)
        c2 = concat(c1,l2)
        l3 = conv1(filters, c2)
        c3 = concat(c2,l3)
        l4 = conv1(filters,c3)

        return concat(input,l4)

    def td_block(conv1_filters,conv2_filters,input):
        TD = conv1(conv1_filters,conv2(conv2_filters,input))
        print("TD Shape = ", TD.shape)
        DB = dense_block(conv1_filters,TD)
        print("Dense Block Shape: ", DB.shape)
        return DB

    def tu_block(conv1_filters,tran2_filters,input,td_input):
        TU = conv1(conv1_filters,tran2(tran2_filters,input))
        print("TU Shape: ", TU.shape)
        C = concat(TU,td_input)
        print("C Shape: ", C.shape)
        DB = dense_block(conv1_filters,C)
        print("Dense Block Shape: ", DB.shape)
        return DB

    #Build Model#
    # TD = convolutions that train down, DB = Dense blocks, TU = Transpose convolutions that train up, C = concatenation groups.

    inputs = Input(shape=(512,512,1))
    print("Input Shape: ", inputs.shape)
    TD1 = conv2(10,inputs) #basic convolution of stride 2, no maxpool.
    print("TD Shape: ", TD1.shape)
    DB1 = dense_block(10,TD1)
    print("Dense Block Shape: ",DB1.shape )

    TD2 = td_block(15,10, DB1)
    TD3 = td_block(20,15,TD2)
    TD4 = td_block(25,20,TD3)
    TD5 = td_block(30,25,TD4)
    print("TD5 shape: ", TD5.shape)

    TU1 = tu_block(25,30,TD5,TD4)
    TU2 = tu_block(20,25,TU1,TD3)
    TU3 = tu_block(15,20,TU2,TD2)
    TU4 = tu_block(10,15,TU3,TD1)
    TU5 = tran2(1, TU4)


    logits = layers.Conv2D(filters = 1,  **kwargs)(TU5)
    print(logits.shape)
    model = Model(inputs=inputs, outputs=logits )


    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001, epsilon=.1)  
    loss = "categorical_crossentropy"
    metrics = ['accuracy',custom.dsc(cls=6),'categorical_crossentropy','sparse_categorical_crossentropy']
    model.compile(optimizer= optimizer,loss = loss, metrics =metrics)
    model.summary()
    '''Train Model'''
    time_start = time.time()
    model.fit_generator(dataset, epochs=10)#batch_size=15, use_multiprocessing=True
    time_end = time.time()
    print("Fitting completed in {} min".format((time_end-time_start)/60))
    send_text("Fitting completed in {} min".format((time_end-time_start)/60))
   
    '''Save Model'''
    if os.path.isdir(save_path) == False:
         os.mkdir(save_path)
    
    model.save(save_path)


if os.path.isfile(save_path) == True:
     model = tf.keras.models.load_model(save_path)
else:
    NN()



Input Shape:  (None, 512, 512, 1)
TD Shape:  (None, 256, 256, 10)
Dense Block Shape:  (None, 256, 256, 20)
TD Shape =  (None, 128, 128, 15)
Dense Block Shape:  (None, 128, 128, 30)
TD Shape =  (None, 64, 64, 20)
Dense Block Shape:  (None, 64, 64, 40)
TD Shape =  (None, 32, 32, 25)
Dense Block Shape:  (None, 32, 32, 50)
TD Shape =  (None, 16, 16, 30)
Dense Block Shape:  (None, 16, 16, 60)
TD5 shape:  (None, 16, 16, 60)
TU Shape:  (None, 32, 32, 25)
C Shape:  (None, 32, 32, 75)
Dense Block Shape:  (None, 32, 32, 100)
TU Shape:  (None, 64, 64, 20)
C Shape:  (None, 64, 64, 60)
Dense Block Shape:  (None, 64, 64, 80)
TU Shape:  (None, 128, 128, 15)
C Shape:  (None, 128, 128, 45)
Dense Block Shape:  (None, 128, 128, 60)
TU Shape:  (None, 256, 256, 10)
C Shape:  (None, 256, 256, 20)
Dense Block Shape:  (None, 256, 256, 30)
(None, 512, 512, 1)
Model: "functional_17"
__________________________________________________________________________________________________
Layer (type)                   

InvalidArgumentError: ignored